In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from torch.nn import Softplus #smooth relu
import torch.nn as nn
import torch.nn.functional as F
from utils import generate_polynomial_data
import numpy as np
import torch
import torch.optim as optim
from torch.nn import MSELoss
import pickle
from tqdm import tqdm

from model import Net, check_loss_landscape

In [24]:
torch.manual_seed(0)
suffix = "4"

n = 50
d = 1000
generate_data = True
usepickle = True

coeffs = -1+2*np.random.rand(d, 1)
xvals = -1+2*np.random.rand(n)
fileend = ".pickle" if usepickle else ".npy"

if generate_data:
    print("Generating Data...")
    X, Y = generate_polynomial_data(coeffs, xvals)
    with open("./datasets/X%s%s" %(("pickle_" if usepickle else ""), suffix)+fileend, "wb") as f:
        if usepickle:
            pickle.dump(X, f)
        else:
            np.save(f, X)
    with open("./datasets/Y%s%s" %(("pickle_" if usepickle else ""),suffix) + fileend, "wb") as f:
        if usepickle:
            pickle.dump(Y, f)
        else:
            np.save(f, Y)
    with open("./datasets/coeffs%s%s" %(("pickle_" if usepickle else ""),suffix) + fileend, "wb") as f:
        if usepickle:
            pickle.dump(coeffs, f)
        else:
            np.save(f, coeffs)
    
else:
    with open("./datasets/X%s%s" %(("pickle_" if usepickle else ""),suffix) + fileend, "rb") as f:
        if usepickle:
            X = pickle.load(f)
        else:
            X = np.load(f)
    with open("./datasets/Y%s%s" %(("pickle_" if usepickle else ""),suffix) + fileend, "rb") as f:
        if usepickle:
            Y = pickle.load(f)
        else:
            Y = np.load(f)
    with open("./datasets/coeffs%s%s" %(("pickle_" if usepickle else ""),suffix) + fileend, "rb") as f:
        if usepickle:
            coeffs = pickle.load(f)
        else:
            coeffs = np.load(f)

100%|██████████| 50/50 [00:00<00:00, 1081.36it/s]

Generating Data...


In [23]:
X

array([[ 1.00000000e+000, -7.04734149e-001,  4.96650220e-001, ...,
        -3.02858786e-152,  2.13434928e-152, -1.50414882e-152],
       [ 1.00000000e+000, -8.78098455e-001,  7.71056896e-001, ...,
        -5.15921053e-057,  4.53029480e-057, -3.97804486e-057],
       [ 1.00000000e+000,  5.79526564e-001,  3.35851038e-001, ...,
         6.08313572e-237,  3.52533874e-237,  2.04302745e-237],
       ...,
       [ 1.00000000e+000, -7.21945981e-001,  5.21206000e-001, ...,
        -8.49550440e-142,  6.13329526e-142, -4.42790786e-142],
       [ 1.00000000e+000,  6.00802081e-002,  3.60963140e-003, ...,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
       [ 1.00000000e+000, -2.83321121e-001,  8.02708575e-002, ...,
        -0.00000000e+000,  0.00000000e+000, -0.00000000e+000]])

In [16]:
model_path = "./models/model_sgd_%s_O100_params.pt"%suffix
net = Net(d, epochs = 50)
train=True

In [18]:
sgd=  True
if train:
    usegpu=True
    if torch.cuda.is_available() and usegpu:
        device = torch.device("cuda:0")
        print("Running on GPU")
    else:
        device = torch.device("cpu")
    net.to(device)
    if sgd:
        net.train_sgd(X, Y, 100000, lr = 1e-2)
    else:
        net.train_gd(X, Y, 2000, lr = 1e-3, usegpu = usegpu)
    torch.save(net.state_dict(), model_path)

  0%|          | 0/50 [00:00<?, ?it/s]

Running on GPU
Running on GPU
0.01
epoch: 0\epochLoss =  0.00049

  0%|          | 0/50 [02:22<?, ?it/s]


KeyboardInterrupt: 